In [1]:
# -*- coding: utf-8 -*-
import tensorflow as tf
import numpy as np
import os
import random
import time
import pickle
import logging
from PIL import Image
import tensorflow.contrib.slim as slim
from tensorflow.python.ops import random_ops,control_flow_ops
from tensorflow.python.keras.initializers import he_normal
import cv2

import sys
add_dir = os.path.abspath("/opt/DahuaTech/MVViewer/share/Python")
sys.path.append(add_dir)
from capture import *

from matplotlib import pyplot as plt
import time

from PyQt5 import QtCore, QtGui, QtWidgets

tf.app.flags.DEFINE_boolean('random_flip_up_down', False, "Whether to random flip up down")
tf.app.flags.DEFINE_boolean('random_brightness', True, "whether to adjust brightness")
tf.app.flags.DEFINE_boolean('random_contrast', True, "whether to random constrast")

tf.app.flags.DEFINE_integer('charset_size', 3, "Choose the first `charset_size` character to conduct our experiment.")
tf.app.flags.DEFINE_integer('image_size', 224, "Needs to provide same value as in training.")
tf.app.flags.DEFINE_boolean('gray', True, "whether to change the rbg to gray")
tf.app.flags.DEFINE_integer('max_steps', 100000, 'the max training steps ')
tf.app.flags.DEFINE_integer('eval_steps', 50, "the step num to eval")
tf.app.flags.DEFINE_integer('save_steps', 2000, "the steps to save")

tf.app.flags.DEFINE_string('checkpoint_dir', '/media/xinje/New Volume/building_blocks/checkpoint/', 'the checkpoint dir')
tf.app.flags.DEFINE_string('train_data_dir', './train/', 'the train dataset dir')
tf.app.flags.DEFINE_string('test_data_dir', './test/', 'the test dataset dir')
tf.app.flags.DEFINE_string('log_dir', './log', 'the logging dir')

tf.app.flags.DEFINE_boolean('restore', False, 'whether to restore from checkpoint')
tf.app.flags.DEFINE_integer('epoch', 1, 'Number of epoches')
tf.app.flags.DEFINE_integer('batch_size', 16, 'Validation batch size')
tf.app.flags.DEFINE_string('mode', 'train', 'Running mode. One of {"train", "valid", "test"}')
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

def fully_connected(prev_layer, num_units, is_training):
    layer = tf.layers.dense(prev_layer, num_units, use_bias=True, activation=None)
    layer = tf.layers.batch_normalization(layer, training=is_training)
    layer = tf.nn.relu(layer)
    return layer
def conv_layer_conv(prev_layer, layer_depth, is_training):
    conv_layer1 = tf.layers.conv2d(prev_layer, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)

    pool_layer1 = tf.layers.max_pooling2d(conv_layer1_out,[2,2],strides=2)
    return pool_layer1
def conv_layer_2conv(prev_layer, layer_depth, is_training):
    conv_layer1 = tf.layers.conv2d(prev_layer, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)
    
    conv_layer2 = tf.layers.conv2d(conv_layer1_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer2_bn = tf.layers.batch_normalization(conv_layer2, training=is_training)
    conv_layer2_out = tf.nn.relu(conv_layer2_bn)

    pool_layer2 = tf.layers.max_pooling2d(conv_layer2_out,[2,2],strides=2)
    return pool_layer2
def conv_layer_3conv(prev_layer, layer_depth, is_training):
    conv_layer1 = tf.layers.conv2d(prev_layer, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)
    
    conv_layer2 = tf.layers.conv2d(conv_layer1_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer2_bn = tf.layers.batch_normalization(conv_layer2, training=is_training)
    conv_layer2_out = tf.nn.relu(conv_layer2_bn)
    
    conv_layer3 = tf.layers.conv2d(conv_layer2_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer3_bn = tf.layers.batch_normalization(conv_layer3, training=is_training)
    conv_layer3_out = tf.nn.relu(conv_layer3_bn)
    pool_layer3 = tf.layers.max_pooling2d(conv_layer3_out,[2,2],strides=2)
    return pool_layer3
def conv_layer_4conv(prev_layer, layer_depth, is_training):
    conv_layer1 = tf.layers.conv2d(prev_layer, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)
    
    conv_layer2 = tf.layers.conv2d(conv_layer1_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer2_bn = tf.layers.batch_normalization(conv_layer2, training=is_training)
    conv_layer2_out = tf.nn.relu(conv_layer2_bn)
    
    conv_layer3 = tf.layers.conv2d(conv_layer2_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer3_bn = tf.layers.batch_normalization(conv_layer3, training=is_training)
    conv_layer3_out = tf.nn.relu(conv_layer3_bn)
    
    conv_layer4 = tf.layers.conv2d(conv_layer3_out, layer_depth, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer4_bn = tf.layers.batch_normalization(conv_layer4, training=is_training)
    conv_layer4_out = tf.nn.relu(conv_layer4_bn)
    pool_layer4 = tf.layers.max_pooling2d(conv_layer4_out,[2,2],strides=2)
    return pool_layer4

def identity_block(layer,layer_depth,is_training):
    depth1,depth2,depth3 = layer_depth
    shortcut = layer
    conv_layer1 = tf.layers.conv2d(layer, depth1, [1,1], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)
    
    conv_layer2 = tf.layers.conv2d(conv_layer1_out, depth2, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer2_bn = tf.layers.batch_normalization(conv_layer2, training=is_training)
    conv_layer2_out = tf.nn.relu(conv_layer2_bn)
    
    conv_layer3 = tf.layers.conv2d(conv_layer2_out, depth3, [1,1], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer3_bn = tf.layers.batch_normalization(conv_layer3, training=is_training)
    
#     conv_temp = tf.layers.conv2d(shortcut,depth3,[1,1],1,'same')
    conv_add = shortcut+conv_layer3_bn
    conv_layer3_out = tf.nn.relu(conv_add)

    return conv_layer3_out

def convolutional_block(layer,layer_depth,is_training):
    depth1,depth2,depth3 = layer_depth
    shortcut = layer
    conv_layer1 = tf.layers.conv2d(layer, depth1, [1,1], 2, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer1_bn = tf.layers.batch_normalization(conv_layer1, training=is_training)
    conv_layer1_out = tf.nn.relu(conv_layer1_bn)
    
    conv_layer2 = tf.layers.conv2d(conv_layer1_out, depth2, [3,3], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer2_bn = tf.layers.batch_normalization(conv_layer2, training=is_training)
    conv_layer2_out = tf.nn.relu(conv_layer2_bn)
    
    conv_layer3 = tf.layers.conv2d(conv_layer2_out, depth3, [1,1], 1, 'same', use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
    conv_layer3_bn = tf.layers.batch_normalization(conv_layer3, training=is_training)
    
    conv_temp = tf.layers.conv2d(shortcut,depth3,[1,1],2,'same')
    conv_temp = tf.layers.batch_normalization(conv_temp,training=is_training)
    conv_add = conv_temp+conv_layer3_bn
    conv_layer3_out = tf.nn.relu(conv_add)
#     pool_layer3 = tf.layers.max_pooling2d(conv_layer3_out,[2,2],strides=2)
    return conv_layer3_out

def connect_camera():
    cameraCnt, cameraList = enumCameras()
    camera = cameraList[0]
    nRet = openCamera(camera)

    streamSourceInfo = GENICAM_StreamSourceInfo()
    streamSourceInfo.channelId = 0
    streamSourceInfo.pCamera = pointer(camera)

    streamSource = pointer(GENICAM_StreamSource())
    nRet = GENICAM_createStreamSource(pointer(streamSourceInfo), byref(streamSource))
    nRet = setSoftTriggerConf(camera)
    nRet = streamSource.contents.startGrabbing(streamSource, c_ulonglong(0), c_int(GENICAM_EGrabStrategy.grabStrartegySequential))
    time.sleep(1)
    return camera, streamSource

def capture_image(camera,streamSource):
    start =time.time()

    # time.sleep(1)
    nRet = grabOne(camera)  
    if( nRet != 0 ):
        print("grabOne fail!")
        # 释放相关资源
        streamSource.contents.release(streamSource)        
    else:
        print("trigger time: " + str(datetime.datetime.now()))

    frame = pointer(GENICAM_Frame())
    nRet = streamSource.contents.getFrame(streamSource, byref(frame), c_uint(1000))
    if ( nRet != 0 ):
        print("SoftTrigger getFrame fail! timeOut [1000]ms")
        # 释放相关资源
        streamSource.contents.release(streamSource)   
    else:
        print("SoftTrigger getFrame success BlockId = " + str(frame.contents.getBlockId(frame))) 
        print("get frame time: " + str(datetime.datetime.now()))   

    nRet = frame.contents.valid(frame)
    if ( nRet != 0 ):
        print("frame is invalid!")
        # 释放驱动图像缓存资源
        frame.contents.release(frame)
        # 释放相关资源
        streamSource.contents.release(streamSource)


    imageSize = frame.contents.getImageSize(frame)
    buffAddr = frame.contents.getImage(frame)
    frameBuff = c_buffer(b'\0', imageSize)
    memmove(frameBuff, c_char_p(buffAddr), imageSize)

    convertParams = IMGCNV_SOpenParam()
    convertParams.dataSize = imageSize
    convertParams.height = frame.contents.getImageHeight(frame)
    convertParams.width = frame.contents.getImageWidth(frame)
    convertParams.paddingX = frame.contents.getImagePaddingX(frame)
    convertParams.paddingY = frame.contents.getImagePaddingY(frame)
    convertParams.pixelForamt = frame.contents.getImagePixelFormat(frame)
    frame.contents.release(frame)

    # 保存bmp图片  
    bmpInfoHeader = BITMAPINFOHEADER() 
    bmpFileHeader = BITMAPFILEHEADER()

    uRgbQuadLen = 0
    rgbQuad = (RGBQUAD * 256)() # 调色板信息
    rgbBuff = c_buffer(b'\0', convertParams.height * convertParams.width * 3)

    # 如果图像格式是 Mono8 不需要转码 
    if convertParams.pixelForamt == EPixelType.gvspPixelMono8:
        # 初始化调色板rgbQuad 实际应用中 rgbQuad 只需初始化一次
        for i in range(0, 256):
            rgbQuad[i].rgbBlue = rgbQuad[i].rgbGreen = rgbQuad[i].rgbRed = i;

        uRgbQuadLen = sizeof(RGBQUAD) * 256    
        bmpFileHeader.bfSize = sizeof(bmpFileHeader) + sizeof(bmpInfoHeader) + uRgbQuadLen + convertParams.dataSize
        bmpInfoHeader.biBitCount = 8
    else:
        # 转码 => BGR24
        rgbSize = c_int()
        nRet = IMGCNV_ConvertToBGR24(cast(frameBuff, c_void_p), byref(convertParams), \
                                     cast(rgbBuff, c_void_p), byref(rgbSize))

        if ( nRet != 0 ):
            print("image convert fail! errorCode = " + str(nRet))
            # 释放相关资源
            streamSource.contents.release(streamSource) 

        bmpFileHeader.bfSize = sizeof(bmpFileHeader) + sizeof(bmpInfoHeader) + rgbSize.value
        bmpInfoHeader.biBitCount = 24   

    if convertParams.pixelForamt == EPixelType.gvspPixelMono8:

        image_information = frameBuff
    else: 
        image_information = rgbBuff
    elapsed = time.time()-start
    print(elapsed)
    return image_information

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(685, 672)
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setMaximumSize(QtCore.QSize(1200, 1000))
        self.centralwidget.setObjectName("centralwidget")
        self.horizontalLayoutWidget = QtWidgets.QWidget(self.centralwidget)
        self.horizontalLayoutWidget.setGeometry(QtCore.QRect(30, 10, 621, 611))
        self.horizontalLayoutWidget.setObjectName("horizontalLayoutWidget")
        self.horizontalLayout = QtWidgets.QHBoxLayout(self.horizontalLayoutWidget)
        self.horizontalLayout.setContentsMargins(0, 0, 0, 0)
        self.horizontalLayout.setObjectName("horizontalLayout")
        self.verticalLayout = QtWidgets.QVBoxLayout()
        self.verticalLayout.setSizeConstraint(QtWidgets.QLayout.SetDefaultConstraint)
        self.verticalLayout.setSpacing(0)
        self.verticalLayout.setObjectName("verticalLayout")
        self.horizontalLayout_2 = QtWidgets.QHBoxLayout()
        self.horizontalLayout_2.setObjectName("horizontalLayout_2")
        self.label_4 = QtWidgets.QLabel(self.horizontalLayoutWidget)
        self.label_4.setMinimumSize(QtCore.QSize(200,100))
#         self.label_4.setFrameShape(QtWidgets.QFrame.Box)
        self.label_4.setObjectName("label_4")
        self.horizontalLayout_2.addWidget(self.label_4)
        
        self.label_2 = QtWidgets.QLabel(self.horizontalLayoutWidget)
#         self.label_2.setMinimumSize(QtCore.QSize(100, 0))
        self.label_2.setMaximumSize(QtCore.QSize(500, 100))
#         self.label_2.setFrameShape(QtWidgets.QFrame.Box)
        self.label_2.setObjectName("label_2")
        self.horizontalLayout_2.addWidget(self.label_2)
        self.verticalLayout.addLayout(self.horizontalLayout_2)
        self.label = QtWidgets.QLabel(self.horizontalLayoutWidget)
        self.label.setMinimumSize(QtCore.QSize(0, 300))
        self.label.setMaximumSize(QtCore.QSize(16777215, 500))
        self.label.setFrameShape(QtWidgets.QFrame.Box)
        self.label.setScaledContents(False)
        self.label.setObjectName("label")
        self.verticalLayout.addWidget(self.label)
#         self.label_4 = QtWidgets.QLabel(self.horizontalLayoutWidget)
#         self.label_4.setMinimumSize(QtCore.QSize(0, 0))
#         self.label_4.setFrameShape(QtWidgets.QFrame.Box)
#         self.label_4.setObjectName("label_4")
        self.progressBar = QtWidgets.QProgressBar(self.horizontalLayoutWidget)
        self.progressBar.setProperty("value", 0)
        self.progressBar.setObjectName("progressBar")
        self.verticalLayout.addWidget(self.progressBar)
        self.horizontalLayout.addLayout(self.verticalLayout)
        self.verticalLayout_2 = QtWidgets.QVBoxLayout()
        self.verticalLayout_2.setObjectName("verticalLayout_2")
        self.label_3 = QtWidgets.QLabel(self.horizontalLayoutWidget)
        self.label_3.setMaximumSize(QtCore.QSize(100, 100))
        self.label_3.setLayoutDirection(QtCore.Qt.LeftToRight)
        self.label_3.setFrameShape(QtWidgets.QFrame.Box)
        self.label_3.setObjectName("label_3")
        self.verticalLayout_2.addWidget(self.label_3)
        self.pushButton_2 = QtWidgets.QPushButton(self.horizontalLayoutWidget)
        self.pushButton_2.setMaximumSize(QtCore.QSize(90, 30))
        self.pushButton_2.setObjectName("pushButton_2")
        self.verticalLayout_2.addWidget(self.pushButton_2)
        self.pushButton = QtWidgets.QPushButton(self.horizontalLayoutWidget)
        self.pushButton.setMaximumSize(QtCore.QSize(90, 30))
        self.pushButton.setObjectName("pushButton")
        self.verticalLayout_2.addWidget(self.pushButton)
        self.horizontalLayout.addLayout(self.verticalLayout_2)
        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 685, 20))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)
        self.logo = QtGui.QPixmap('./logo.jpg')
        self.logo = self.logo.scaled(self.label_4.width(), self.label_4.height(),QtCore.Qt.KeepAspectRatio)
        self.ok_pixmap = QtGui.QPixmap('/home/xinje/Desktop/Billy-8.14/Mask_RCNN/samples/balloon/res/icon.jpeg')
        pixmap = QtGui.QPixmap('/home/xinje/Desktop/Billy-8.14/Mask_RCNN/samples/balloon/res/icon.jpeg')
        pixmap = pixmap.scaled(850, 450,QtCore.Qt.KeepAspectRatio)
        self.label_4.setPixmap(self.logo)
        self.label.setAlignment(QtCore.Qt.AlignLeft)
        self.label.setPixmap(pixmap)
        self.ok_pixmap = self.ok_pixmap.scaled(self.label_3.width(), self.label_3.height(),QtCore.Qt.KeepAspectRatio)
        self.label_3.setPixmap(pixmap)
        self.label.setAlignment(QtCore.Qt.AlignCenter)
        self.label_2.setFont(QtGui.QFont("Roman times",10,QtGui.QFont.Bold))
        
        self.pushButton_2.clicked.connect(self.load_model)
        self.pushButton.clicked.connect(self.Inference)
        
        #varibles about Inference
        self.inputs = None
        self.logits = None
        self.is_training = None
        self.sess =None
        self.predict = None
        self.ckpt = None
        self.keep_prob = None
        self.ok_count = 0
        self.ng_count = 0
        self.total_count = 0
        self.ok_pixmap = None
        self.ng_pixmap = None
        self.camera = None
        self.streamSource = None
        
    def reset_variables(self):
        #neural
        self.inputs = None
        self.logits = None
        self.is_training = None
        self.sess =None
        self.predict = None
        self.ckpt = None
        self.keep_prob = 1.0
        #count
        self.ok_count = 0
        self.ng_count = 0
        self.total_count =0
        #camera
        self.camera = None
        self.streamSource = None

    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "XINJE-AI"))
        self.label_2.setText(_translate("MainWindow", "Total: 0       OK: 0       NG: 0"))
        self.label.setText(_translate("MainWindow", "Pic"))
#         self.label_4.setText(_translate("MainWindow", "Canvas"))
        self.label_3.setText(_translate("MainWindow", "Result"))
        self.pushButton_2.setText(_translate("MainWindow", "Load_model"))
        self.pushButton.setText(_translate("MainWindow", "Inference"))
    
#     def CV2QImage(self,cv_image):
#         width = cv_image.shape[1]
#         height = cv_image.shape[0]
#         pixmap = QtGui.QPixmap(width,height)
#         qimg = pixmap.toImage()
        
#         for row in range(0,height):
#             for col in range(0,width):
#                 r = cv_image[row,col,0]
#                 g = cv_image[row,col,1]
#                 b = cv_image[row,col,2]
#                 pix = QtGui.qRgb(r,g,b)
#                 qimg.setPixel(col,row,pix)
#         return qimg

        
    def load_model(self):
        if(self.sess != None):
            QtWidgets.QMessageBox.information(QtWidgets.QWidget().move(600,500),"Warning","You have already loaded the model!",QtWidgets.QMessageBox.Ok)
#             self.sess.close()
#             self.reset_variables()
#             tf.reset_default_graph()
        
        #load_ok/ng img
        else:
            self.camera, self.streamSource = connect_camera()
            self.ok_pixmap = QtGui.QPixmap('./ok.png')
            self.ok_pixmap = self.ok_pixmap.scaled(self.label_3.width(), self.label_3.height(),QtCore.Qt.KeepAspectRatio)
            self.ng_pixmap = QtGui.QPixmap('./ng.png')
            self.ng_pixmap = self.ng_pixmap.scaled(self.label_3.width(), self.label_3.height(),QtCore.Qt.KeepAspectRatio)
            self.inputs = tf.placeholder(tf.float32, [None, 416, 416, 3])
            #labels = tf.placeholder(tf.int64, [None])
            self.keep_prob = tf.placeholder('float')
            self.is_training = tf.placeholder(tf.bool)
            layer = self.inputs

#             conv_layer1 = conv_layer_2conv(layer, 64, self.is_training)
#             conv_layer2 = conv_layer_2conv(conv_layer1, 128, self.is_training)
#             conv_layer3 = conv_layer_3conv(conv_layer2, 256, self.is_training)
#             conv_layer4 = conv_layer_3conv(conv_layer3, 512, self.is_training)
#             conv_layer5 = conv_layer_3conv(conv_layer4, 512, self.is_training)

        #stage1
            X = tf.layers.conv2d(layer,64,[7,7],2,'valid',use_bias=True,kernel_initializer=he_normal(seed=0.01),activation=None,kernel_regularizer=tf.contrib.layers.l2_regularizer(0.01))
            X = tf.layers.batch_normalization(X,training=self.is_training)
            X = tf.nn.relu(X)
            X = tf.layers.max_pooling2d(X,[3,3],strides=2)
        #stage2    
            X = convolutional_block(X,[64,64,256],self.is_training)
            X = identity_block(X,[64,64,256],self.is_training)
            X = identity_block(X,[64,64,256],self.is_training)

        #stage3
            X = convolutional_block(X,[128,128,512],self.is_training)
            X = identity_block(X,[128,128,512],self.is_training)
            X = identity_block(X,[128,128,512],self.is_training)
            X = identity_block(X,[128,128,512],self.is_training)

        #stage4
            X = convolutional_block(X,[256,256,1024],self.is_training)
            X = identity_block(X,[256,256,1024],self.is_training)
            X = identity_block(X,[256,256,1024],self.is_training)
            X = identity_block(X,[256,256,1024],self.is_training)
            X = identity_block(X,[256,256,1024],self.is_training)
            X = identity_block(X,[256,256,1024],self.is_training)

        #stage5
            X = convolutional_block(X,[512,512,2048],self.is_training)
            X = identity_block(X,[512,512,2048],self.is_training)
            X = identity_block(X,[512,512,2048],self.is_training)
        #average_pooling
            X = tf.layers.average_pooling2d(X,[2,2],strides=2)
        # Flatten the output from the convolutional layers
            orig_shape = X.get_shape().as_list()
            layer = tf.reshape(X, shape=[-1, orig_shape[1]*orig_shape[2]*orig_shape[3]])
            layer_drop = tf.nn.dropout(layer,self.keep_prob)
            layer2 = fully_connected(layer_drop, 1000, self.is_training)
            layer_drop2 = tf.nn.dropout(layer2,self.keep_prob)
            self.logits = tf.layers.dense(layer_drop2, 2)


            # Flatten the output from the convolutional layers
#             orig_shape = conv_layer5.get_shape().as_list()
#             layer = tf.reshape(conv_layer5, shape=[-1, orig_shape[1]*orig_shape[2]*orig_shape[3]])
#             self.keep_prob = tf.placeholder('float')
#             layer_drop = tf.nn.dropout(layer,self.keep_prob)
#             layer = fully_connected(layer_drop, 4096, self.is_training)
#             layer_drop2 = tf.nn.dropout(layer,self.keep_prob)
#             layer2 = fully_connected(layer_drop2, 4096, self.is_training)
#             layer_drop3 = tf.nn.dropout(layer2,self.keep_prob)
#             self.logits = tf.layers.dense(layer_drop3, 2)
            self.predict = tf.argmax(self.logits, 1)

            print('Strart Inference:')
    #         self.listWidget.addItem('Strart Inference:')
            self.sess = tf.InteractiveSession()
            #with tf.Session() as sess:

            saver = tf.train.Saver()
            self.sess.run(tf.global_variables_initializer())

            ckpt = tf.train.latest_checkpoint(FLAGS.checkpoint_dir)
            if ckpt:
                saver.restore(self.sess,ckpt)
                print('restore from the checkpoint {0}'.format(ckpt))
    #             self.listWidget.addItem('restore from the checkpoint '+str(ckpt))            
            
    def Inference(self):
        if(self.sess!=None):
#             self.listWidget.addItem('Start inference!')
            start = time.time()
            image_info = capture_image(self.camera,self.streamSource)
            elapsed = (time.time()-start)
            print(elapsed)
            #self.listWidget.add_item('Elapsed '+str(elapsed)+' seconds for inference')
            self.progressBar.setProperty("value", 20)
            image = np.array(list(image_info.raw))
            image = image.reshape([1024,1280,3])
            image = image[:,:,::-1]    
            image = image.astype(np.uint8)
            image1 = cv2.resize(image,(416,416))
            image1 = image1/255.
            test_images = np.expand_dims(image1,axis=0)
            self.progressBar.setProperty("value", 40)
            
            start = time.time()
            pt = self.sess.run([self.predict], feed_dict={self.inputs: test_images,self.is_training: False,self.keep_prob:1.0})
            elapsed = (time.time()-start)
            print('Inference elapsed '+str(elapsed)+' seconds')
            
            if(pt[0][0] == 0):
                print('OK!')
                cv2.putText(image,'OK!',(40,100),cv2.FONT_HERSHEY_PLAIN,8.0,(0,255,0),thickness =2)
                self.label_3.setPixmap(self.ok_pixmap)
                self.ok_count+=1
                self.total_count+=1
                self.label_2.setText("Total: "+str(self.total_count)+"       OK: "+str(self.ok_count)+"       NG: "+str(self.ng_count))
                
#                 self.listWidget.addItem('Result: OK!')
            else:
                print('NG!')
                cv2.putText(image,'NG!',(40,100),cv2.FONT_HERSHEY_PLAIN,8.0,(255,0,0),thickness =2)
                self.label_3.setPixmap(self.ng_pixmap)
                self.ng_count+=1
                self.total_count+=1
                self.label_2.setText("Total: "+str(self.total_count)+"       OK: "+str(self.ok_count)+"       NG: "+str(self.ng_count))
            self.progressBar.setProperty("value", 80)

            
            temp = QtGui.QImage(image[:],image.shape[1],image.shape[0],image.shape[1]*3,QtGui.QImage.Format_RGB888)

            pixmap = QtGui.QPixmap(temp)
            pixmap = pixmap.scaled(self.label.width(), self.label.height(),QtCore.Qt.KeepAspectRatio)
            self.progressBar.setProperty("value", 100)
            self.label.setPixmap(pixmap)
            self.label.setAlignment(QtCore.Qt.AlignCenter)
            
        else:
            alert = QtWidgets.QMessageBox.information(QtWidgets.QWidget().move(600,500),"Warning","Please load the model before Inference!",QtWidgets.QMessageBox.Ok)
            
#             self.listWidget.addItem('Please load the model before Inference!')

In [ ]:
if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

cameraCnt: 1
camera connect success.
Strart Inference:
INFO:tensorflow:Restoring parameters from /media/xinje/New Volume/building_blocks/checkpoint/CH-reco-400
restore from the checkpoint /media/xinje/New Volume/building_blocks/checkpoint/CH-reco-400
trigger time: 2018-10-25 08:35:34.633653
SoftTrigger getFrame success BlockId = 1
get frame time: 2018-10-25 08:35:34.654229
0.027882099151611328
0.027921676635742188
Inference elapsed 0.7661538124084473 seconds
OK!
trigger time: 2018-10-25 08:35:40.567175
SoftTrigger getFrame success BlockId = 2
get frame time: 2018-10-25 08:35:40.587710
0.024869441986083984
0.024904966354370117
Inference elapsed 0.009625911712646484 seconds
NG!
trigger time: 2018-10-25 08:35:59.695281
SoftTrigger getFrame success BlockId = 3
get frame time: 2018-10-25 08:35:59.715770
0.02469491958618164
0.02473139762878418
Inference elapsed 0.011531591415405273 seconds
NG!
trigger time: 2018-10-25 08:36:16.327124
SoftTrigger getFrame success BlockId = 4
get frame time: 2

Inference elapsed 0.010418891906738281 seconds
OK!
trigger time: 2018-10-25 08:37:04.919350
SoftTrigger getFrame success BlockId = 38
get frame time: 2018-10-25 08:37:04.939783
0.024449825286865234
0.024477243423461914
Inference elapsed 0.009768486022949219 seconds
OK!
trigger time: 2018-10-25 08:37:06.407267
SoftTrigger getFrame success BlockId = 39
get frame time: 2018-10-25 08:37:06.427905
0.029855012893676758
0.02988576889038086
Inference elapsed 0.009510040283203125 seconds
OK!
trigger time: 2018-10-25 08:37:07.551346
SoftTrigger getFrame success BlockId = 40
get frame time: 2018-10-25 08:37:07.571973
0.029768943786621094
0.02981281280517578
Inference elapsed 0.009465217590332031 seconds
OK!
trigger time: 2018-10-25 08:37:08.727394
SoftTrigger getFrame success BlockId = 41
get frame time: 2018-10-25 08:37:08.748000
0.03898453712463379
0.039124488830566406
Inference elapsed 0.009139776229858398 seconds
OK!
trigger time: 2018-10-25 08:37:11.623396
SoftTrigger getFrame success BlockI

Inference elapsed 0.009855031967163086 seconds
OK!
trigger time: 2018-10-25 08:38:09.463208
SoftTrigger getFrame success BlockId = 76
get frame time: 2018-10-25 08:38:09.483839
0.02992105484008789
0.029951810836791992
Inference elapsed 0.009818077087402344 seconds
OK!
trigger time: 2018-10-25 08:38:10.943276
SoftTrigger getFrame success BlockId = 77
get frame time: 2018-10-25 08:38:10.963896
0.029834508895874023
0.029866933822631836
Inference elapsed 0.00983572006225586 seconds
OK!
trigger time: 2018-10-25 08:38:12.647326
SoftTrigger getFrame success BlockId = 78
get frame time: 2018-10-25 08:38:12.667986
0.029825925827026367
0.02985668182373047
Inference elapsed 0.009433746337890625 seconds
OK!
trigger time: 2018-10-25 08:38:14.943327
SoftTrigger getFrame success BlockId = 79
get frame time: 2018-10-25 08:38:14.963987
0.029822111129760742
0.029852867126464844
Inference elapsed 0.009426593780517578 seconds
OK!
trigger time: 2018-10-25 08:38:16.543336
SoftTrigger getFrame success BlockI

Inference elapsed 0.00997304916381836 seconds
OK!
trigger time: 2018-10-25 08:39:25.191356
SoftTrigger getFrame success BlockId = 114
get frame time: 2018-10-25 08:39:25.211926
0.03833436965942383
0.03843426704406738
Inference elapsed 0.009817123413085938 seconds
OK!
trigger time: 2018-10-25 08:39:26.199309
SoftTrigger getFrame success BlockId = 115
get frame time: 2018-10-25 08:39:26.219874
0.038671255111694336
0.03877663612365723
Inference elapsed 0.00997781753540039 seconds
OK!
trigger time: 2018-10-25 08:39:27.295308
SoftTrigger getFrame success BlockId = 116
get frame time: 2018-10-25 08:39:27.315939
0.02982473373413086
0.029854297637939453
Inference elapsed 0.00945281982421875 seconds
NG!
trigger time: 2018-10-25 08:39:28.415295
SoftTrigger getFrame success BlockId = 117
get frame time: 2018-10-25 08:39:28.435927
0.029886960983276367
0.029918670654296875
Inference elapsed 0.009484052658081055 seconds
OK!
trigger time: 2018-10-25 08:39:29.767325
SoftTrigger getFrame success BlockI

In [ ]:
a = capture_image()

In [ ]:
len(a)